In [ ]:
!pip install -q kaggle
!pip install -q tensorflow_addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d obulisainaren/retinal-oct-c8

Dataset URL: https://www.kaggle.com/datasets/obulisainaren/retinal-oct-c8
License(s): CC-BY-NC-SA-4.0


In [ ]:
!unzip -q retinal-oct-c8.zip -d data

In [ ]:
base_dir = '/content/data/RetinalOCT_Dataset/RetinalOCT_Dataset'
train_dir = f'{base_dir}/train'
val_dir = f'{base_dir}/val'
test_dir = f'{base_dir}/test'

In [ ]:
img_size = 224
batch_size = 32
classes = ['AMD', 'CNV', 'CSR', 'DME', 'DR', 'DRUSEN', 'MH', 'normal']

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 18400 images belonging to 8 classes.
Found 2800 images belonging to 8 classes.


In [ ]:
print(f"Training classes: {train_generator.class_indices}")
print(f"Validation classes: {val_generator.class_indices}")

Training classes: {'AMD': 0, 'CNV': 1, 'CSR': 2, 'DME': 3, 'DR': 4, 'DRUSEN': 5, 'MH': 6, 'NORMAL': 7}
Validation classes: {'AMD': 0, 'CNV': 1, 'CSR': 2, 'DME': 3, 'DR': 4, 'DRUSEN': 5, 'MH': 6, 'NORMAL': 7}


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='softmax')  # 8 output neurons for 8 classes
])


In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint('best_oct_model.h5', save_best_only=True)
]

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks
)

Epoch 1/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step - accuracy: 0.3385 - loss: 1.7243

575/575 ━━━━━━━━━━━━━━━━━━━━ 303s 499ms/step - accuracy: 0.3387 - loss: 1.7237 - val_accuracy: 0.7768 - val_loss: 0.6717
Epoch 2/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step - accuracy: 0.6241 - loss: 0.9503

575/575 ━━━━━━━━━━━━━━━━━━━━ 280s 486ms/step - accuracy: 0.6242 - loss: 0.9502 - val_accuracy: 0.8050 - val_loss: 0.5116
Epoch 3/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step - accuracy: 0.6974 - loss: 0.7791

575/575 ━━━━━━━━━━━━━━━━━━━━ 321s 484ms/step - accuracy: 0.6974 - loss: 0.7791 - val_accuracy: 0.8218 - val_loss: 0.4638
Epoch 4/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step - accuracy: 0.7245 - loss: 0.6916

575/575 ━━━━━━━━━━━━━━━━━━━━ 280s 487ms/step - accuracy: 0.7245 - loss: 0.6916 - val_accuracy: 0.8479 - val_loss: 0.3873
Epoch 5/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step - accuracy: 0.7470 - loss: 0.6439

575/575 ━━━━━━━━━━━━━━━━━━━━ 319s 482ms/step - accuracy: 0.7470 - loss: 0.6439 - val_accuracy: 0.8479 - val_loss: 0.3849
Epoch 6/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 280s 486ms/step - accuracy: 0.7668 - loss: 0.6058 - val_accuracy: 0.8429 - val_loss: 0.3993
Epoch 7/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 278s 483ms/step - accuracy: 0.7737 - loss: 0.5854 - val_accuracy: 0.8296 - val_loss: 0.4131
Epoch 8/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step - accuracy: 0.7810 - loss: 0.5668

575/575 ━━━━━━━━━━━━━━━━━━━━ 278s 483ms/step - accuracy: 0.7810 - loss: 0.5668 - val_accuracy: 0.8446 - val_loss: 0.3797
Epoch 9/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step - accuracy: 0.7894 - loss: 0.5394

575/575 ━━━━━━━━━━━━━━━━━━━━ 321s 482ms/step - accuracy: 0.7894 - loss: 0.5394 - val_accuracy: 0.8696 - val_loss: 0.3408
Epoch 10/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 284s 494ms/step - accuracy: 0.7907 - loss: 0.5452 - val_accuracy: 0.8543 - val_loss: 0.3595
Epoch 11/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 276s 480ms/step - accuracy: 0.7981 - loss: 0.5223 - val_accuracy: 0.8457 - val_loss: 0.3917
Epoch 12/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 274s 477ms/step - accuracy: 0.8079 - loss: 0.4971 - val_accuracy: 0.8454 - val_loss: 0.3764
